In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

2024-03-03 11:57:02.026450: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-03 11:57:02.066765: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-03 11:57:02.066798: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-03 11:57:02.067845: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-03 11:57:02.075011: I tensorflow/core/platform/cpu_feature_guar

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:

import numpy as np
import pandas as pd
import cv2
import os
from pathlib import Path
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib.pyplot import imread, imshow, subplots, show
%matplotlib inline

In [4]:
# Params
from dataclasses import dataclass

@dataclass
class Params :
    TRAIN_FOLDER_PATH : str = '../data/train'
    TEST_FOLDER_PATH : str = '../data/test'
    BATCH_SIZE : int = 32
    INPUT_HEIGHT : int = 120
    INPUT_WIDTH : int = 100
    CLASSES : int = 32
    EPOCHS : int = 200

# Reset Test folder

In [5]:
from helpers import destroy_test_dir

In [6]:
destroy_test_dir(Params.TEST_FOLDER_PATH)

../data/test/k7el_rey/20240226_183858.jpg => ../data/train/k7el_rey/20240226_183858.jpg
../data/test/k7el_rey/20240226_183608.jpg => ../data/train/k7el_rey/20240226_183608.jpg
../data/test/k7el_rey/20240226_183919.jpg => ../data/train/k7el_rey/20240226_183919.jpg
../data/test/k7el_rey/20240226_184010.jpg => ../data/train/k7el_rey/20240226_184010.jpg
../data/test/k7el_rey/20240226_183118.jpg => ../data/train/k7el_rey/20240226_183118.jpg
../data/test/k7el_rey/20240226_183604.jpg => ../data/train/k7el_rey/20240226_183604.jpg
../data/test/k7el_rey/20240226_183357.jpg => ../data/train/k7el_rey/20240226_183357.jpg
../data/test/k7el_rey/20240226_183135.jpg => ../data/train/k7el_rey/20240226_183135.jpg
../data/test/k7el_rey/20240226_183521.jpg => ../data/train/k7el_rey/20240226_183521.jpg
../data/test/k7el_rey/20240226_183912.jpg => ../data/train/k7el_rey/20240226_183912.jpg
../data/test/copas_las/IMG_20240226_121814.jpg => ../data/train/copas_las/IMG_20240226_121814.jpg
../data/test/copas_las

# Data checks and Splits

In [7]:
def check_image_sizes(folder_path):
    # Get all subfolders and their contents
    folder = Path(folder_path)
    subfolders = [subfolder for subfolder in folder.iterdir() if subfolder.is_dir()]
    counter = 0
    # Get the size of the first image found
    first_image_size = None
    for subfolder in subfolders:
        images = list(subfolder.glob('*.jpg'))
        if images:
            with Image.open(images[0]) as img:
                first_image_size = img.size

            for image_path in images:
                with Image.open(image_path) as img:
                    if img.size != first_image_size:
                        # print(f"Image {image_path.name} in folder {subfolder.name} does not have the same size.")
                        counter += 1

    if counter != 0:
        print(f'{counter} images have a different size')
        return False
    print("All images have the same size.")
    return True

check_image_sizes(folder_path = Params.TRAIN_FOLDER_PATH)

956 images have a different size


/home/yjalil/.pyenv/versions/3.11.5/envs/globalEnv/lib/python3.11/site-packages/PIL/TiffImagePlugin.py:868: UserWarning: Truncated File Read
  warnings.warn(str(msg))
/home/yjalil/.pyenv/versions/3.11.5/envs/globalEnv/lib/python3.11/site-packages/PIL/JpegImagePlugin.py:846: UserWarning: Image appears to be a malformed MPO file, it will be interpreted as a base JPEG file
  warnings.warn(


False

In [8]:
from helpers import create_test_dir

In [9]:
create_test_dir(Params.TRAIN_FOLDER_PATH, Params.TEST_FOLDER_PATH, 0.1)

Test dataset created successfully.


In [10]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Resizing(Params.INPUT_HEIGHT,Params.INPUT_WIDTH),
    tf.keras.layers.Conv2D(64, (2, 2), activation="relu", padding="same"),
    tf.keras.layers.MaxPooling2D((2, 2)),

    tf.keras.layers.Conv2D(128, (2, 2), activation="relu", padding="same"),
    tf.keras.layers.MaxPooling2D((2, 2)),

    tf.keras.layers.Conv2D(256, (2, 2), activation="relu", padding="same"),
    tf.keras.layers.MaxPooling2D((2, 2)),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),

    tf.keras.layers.Dense(Params.CLASSES, activation='softmax')
])

model.compile('adam', loss='categorical_crossentropy', metrics=['accuracy'])


2024-03-03 11:57:06.634079: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-03 11:57:06.634171: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-03 11:57:06.634190: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-03 11:57:06.777425: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-03 11:57:06.777481: I external/local_xla/xla/stream_executor

In [11]:
train_ds, val_ds = tf.keras.utils.image_dataset_from_directory(
  Path(Params.TRAIN_FOLDER_PATH),
  validation_split=0.2,
  subset="both",
  labels='inferred',
  label_mode='categorical',
  seed=123,
  batch_size=Params.BATCH_SIZE)

Found 2879 files belonging to 32 classes.
Using 2304 files for training.
Using 575 files for validation.


In [12]:
test_ds = tf.keras.utils.image_dataset_from_directory(
  Path(Params.TEST_FOLDER_PATH),
  labels='inferred',
  label_mode='categorical',
  batch_size=Params.BATCH_SIZE)

Found 320 files belonging to 32 classes.


In [13]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

history = model.fit(
        train_ds,
        epochs=Params.EPOCHS,
        callbacks = [tf.keras.callbacks.EarlyStopping(patience = 5, verbose = 2)],
        validation_data=val_ds)

Epoch 1/200


2024-03-03 11:57:12.032270: I external/local_xla/xla/service/service.cc:168] XLA service 0x55b271a91c90 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-03-03 11:57:12.032332: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Laptop GPU, Compute Capability 8.6
2024-03-03 11:57:12.126756: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-03-03 11:57:12.556908: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904


 2/72 ━━━━━━━━━━━━━━━━━━━━ 3s 54ms/step - accuracy: 0.0234 - loss: 92.2036  

I0000 00:00:1709463438.481319  151143 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


72/72 ━━━━━━━━━━━━━━━━━━━━ 29s 267ms/step - accuracy: 0.0381 - loss: 33.6086 - val_accuracy: 0.0748 - val_loss: 3.3564
Epoch 2/200
72/72 ━━━━━━━━━━━━━━━━━━━━ 18s 231ms/step - accuracy: 0.1257 - loss: 3.2086 - val_accuracy: 0.1670 - val_loss: 3.0207
Epoch 3/200
72/72 ━━━━━━━━━━━━━━━━━━━━ 18s 227ms/step - accuracy: 0.2147 - loss: 2.7374 - val_accuracy: 0.2278 - val_loss: 2.8518
Epoch 4/200
72/72 ━━━━━━━━━━━━━━━━━━━━ 18s 229ms/step - accuracy: 0.3517 - loss: 2.2451 - val_accuracy: 0.2383 - val_loss: 3.0046
Epoch 5/200
72/72 ━━━━━━━━━━━━━━━━━━━━ 20s 249ms/step - accuracy: 0.4325 - loss: 1.9168 - val_accuracy: 0.3791 - val_loss: 2.3104
Epoch 6/200
72/72 ━━━━━━━━━━━━━━━━━━━━ 18s 231ms/step - accuracy: 0.5497 - loss: 1.3808 - val_accuracy: 0.4504 - val_loss: 2.1123
Epoch 7/200
72/72 ━━━━━━━━━━━━━━━━━━━━ 20s 222ms/step - accuracy: 0.6871 - loss: 0.9650 - val_accuracy: 0.4557 - val_loss: 2.5902
Epoch 8/200
72/72 ━━━━━━━━━━━━━━━━━━━━ 18s 224ms/step - accuracy: 0.7170 - loss: 0.8143 - val_accurac

In [22]:
def plot_history(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']

    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs_range = range(Params.Epochs)

    plt.figure(figsize=(8, 8))
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, acc, label='Training Accuracy')
    plt.plot(epochs_range, val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.title('Training and Validation Accuracy')

    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, loss, label='Training Loss')
    plt.plot(epochs_range, val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')
    plt.show()

In [23]:
plot_history(history)

NameError: name 'epochs' is not defined

In [20]:
result = model.evaluate(test_ds)

10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 52ms/step - accuracy: 0.5726 - loss: 2.3924


[2.525519371032715, 0.559374988079071]